In [2]:
import pandas as pd
import os
import datetime


In [4]:
import pandas as pd
import os

df = pd.read_excel('C:/Users/HP/Downloads/EV업데이트 0729 1846.xlsx')

except_list = ['승인불가', '신청포기', '지원취소']

df = df[~df['신청단계'].isin(except_list)]
df = df[df['신청일자'] == '2025-07-29']

df.shape


(74, 70)

## 파이프라인 EV 미신청건 확인

In [7]:
df = pd.read_excel('Q3.xlsx', sheet_name='PipeLine')
df_gs = pd.read_excel('Greet_Subsidy.xlsx', sheet_name='DOA 박민정', header=1)

# 'Greet Note' 컬럼에 결측치가 있을 수 있으므로, 결측치를 빈 문자열로 대체 후 필터링
df_merge = pd.merge(df, df_gs[['RN', 'Greet Note']], on='RN', how='left')
df_merge['Greet Note'] = df_merge['Greet Note'].fillna('')
df_merge = df_merge[~df_merge['Greet Note'].str.contains('#')]
df_merge.to_excel('11.xlsx', index=False)

c:\Users\HP\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


In [21]:
df = pd.read_excel('Q3.xlsx', sheet_name='PipeLine')
df_complete = pd.read_excel('C:/Users/HP/Downloads/EV업데이트 0731 1819.xlsx')
df_fail = pd.read_excel('Q3.xlsx', sheet_name='미신청건')
except_list = ['승인불가', '신청포기', '지원취소']

fail_rn = df_fail['RN'].tolist()

success_rn = df[~df['RN'].isin(fail_rn)]['RN'].tolist()

# '제조수입사\n관리번호'가 1개만 존재하는 경우만 필터링
single_rn = df_complete['제조수입사\n관리번호'].value_counts()
single_rn = single_rn[single_rn == 1].index.tolist()

df_complete[
    (df_complete['제조수입사\n관리번호'].isin(success_rn)) &
    (df_complete['신청단계'].isin(except_list)) &
    (df_complete['제조수입사\n관리번호'].isin(single_rn))
][['신청단계', '제조수입사\n관리번호', '신청일자', '취소(포기)\n일자']]




,신청단계,제조수입사\n관리번호,신청일자,취소(포기)\n일자
1,신청포기,RN124274641,2025-07-03,2025-07-10
9,신청포기,RN124163310,2025-07-03,2025-07-10
14,신청포기,RN124162908,2025-07-03,2025-07-10
61,신청포기,RN125299678,2025-07-25,2025-07-30
77,신청포기,RN124466671,2025-07-08,2025-07-11
...,...,...,...,...
3997,지원취소,RN124383883,2025-07-16,2025-07-17
4023,지원취소,RN124372536,2025-07-16,2025-07-17
4055,신청포기,RN124446304,2025-07-16,2025-07-30
4061,지원취소,RN125179472,2025-07-16,2025-07-22


In [24]:
df = pd.read_excel('Q3.xlsx', sheet_name='미신청건')
df_g = pd.read_excel('Greet_Subsidy.xlsx', sheet_name='DOA 박민정', header=1)
df_g = df_g[['RN', 'Greet Note']]

fail_rn = df['RN'].tolist()

df_g[df_g['RN'].isin(fail_rn)][['RN', 'Greet Note']].to_excel('11.xlsx', index=False)

c:\Users\HP\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


## 지원 미신청 사유

In [36]:
# 날짜와 상관없이 전체 데이터를 저장합니다.

df_g = pd.read_excel('Greet_Subsidy.xlsx', sheet_name='DOA 박민정', header=1)
df_merge = pd.read_excel('Q3.xlsx', sheet_name='PipeLine')
df_merge = pd.merge(df_merge, df_g[['RN', 'Greet Note']], on='RN', how='left')

# '날짜' 컬럼이 datetime이 아닐 수 있으니 변환
df_merge['날짜'] = pd.to_datetime(df_merge['날짜'], errors='coerce').dt.date

df_merge.to_excel('미신청건.xlsx', index=False)


c:\Users\HP\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


In [35]:
df_merge = pd.read_excel('Q3.xlsx', sheet_name='미신청건')
df_g = pd.read_excel('Greet_Subsidy.xlsx', sheet_name='DOA 박민정', header=1)

# 'Greet Note' 컬럼명이 실제로 존재하는지 확인하고, 없으면 컬럼명을 출력해서 안내
if 'Greet Note' not in df_g.columns:
    print("Greet_Subsidy.xlsx의 'DOA 박민정' 시트 컬럼 목록:", df_g.columns.tolist())
    raise KeyError("'Greet Note' 컬럼이 존재하지 않습니다. 실제 컬럼명을 확인하세요.")

# RN 기준으로 merge
df_merge = pd.merge(df_merge, df_g[['RN', 'Greet Note']], on='RN', how='left')

# 'Greet Note' 컬럼이 없는 경우를 대비하여 fillna 처리
df_merge['Greet Note'] = df_merge['Greet Note'].fillna('')

# Greet Note에 '#'이 없는 것만 필터링
df_filtered = df_merge[~df_merge['Greet Note'].astype(str).str.contains('#')]

# 날짜, RN, Greet Note만 추출하여 저장
df_filtered[['날짜', 'RN', 'Greet Note']].to_excel('미신청건.xlsx', index=False)

c:\Users\HP\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


KeyError: 'Greet Note'

In [ ]:
df = pd.read_excel('C:/Users/HP/Downloads/EV업데이트 0730 1803.xlsx')

except_list = ['승인불가', '신청포기', '지원취소']

df = df[df['신청단계'].isin(except_list)]
df = df[df['취소(포기)\n일자'] == '2025-07-30']

df['신청단계'].value_counts()

신청단계
신청포기    4
지원취소    3
Name: count, dtype: int64

In [5]:
df = pd.read_excel('C:/Users/HP/Downloads/EV업데이트 0730 1803.xlsx')

except_list = ['승인불가', '신청포기', '지원취소']

df = df[pd.to_datetime(df['지급신청일자'], errors='coerce').dt.strftime('%Y-%m-%d') == '2025-07-30']

df.shape

(196, 70)

In [3]:
df = pd.read_excel('Greet_Subsidy.xlsx', sheet_name='지급신청', header=3)

df = df[df['배분일'] == '2025-07-30']

df.shape

(190, 41)

In [12]:
df = pd.read_excel('C:/Users/HP/Downloads/OneDrive_2025-07-30/EV업데이트 0714 1800.xls', header=2)

except_list = ['승인불가', '신청포기', '지원취소']

df = df[~df['신청단계'].isin(except_list)]
df = df[df['신청일자'] == '2025-07-14']

df.shape

(130, 70)

In [ ]:
df = 

In [7]:
# '신청단계' KeyError가 발생하는 이유는 파일마다 컬럼명이 다르거나, 공백/개행/숨은문자 등이 포함되어 있을 수 있기 때문입니다.
# 아래처럼 컬럼명을 strip()으로 정리하고, 실제 '신청단계' 컬럼이 있는지 확인 후 처리하면 안전합니다.

import os
import pandas as pd

folder_path = 'C:/Users/HP/Downloads/OneDrive_2025-07-31 (3)'
except_list = ['승인불가', '신청포기', '지원취소']

날짜_개수_리스트 = []

for file in os.listdir(folder_path):
    if file.endswith('.xlsx'):
        df = pd.read_excel(os.path.join(folder_path, file))
    elif file.endswith('.xls'):
        df = pd.read_excel(os.path.join(folder_path, file), header=2)
    else:
        continue

    # 컬럼명 공백/개행 제거
    df.columns = df.columns.str.strip().str.replace('\n', '', regex=False)
    print(df.columns)

    # '신청단계' 컬럼이 실제로 존재하는지 확인
    if '신청단계' in df.columns:
        df = df[~df['신청단계'].isin(except_list)]
    else:
        print(f"파일 {file}에 '신청단계' 컬럼이 없습니다.")
        continue

    # 파일명에서 날짜 추출
    try:
        date_str = file.split()[3]  # 예: '0724' 또는 '0624' 추출
        month = date_str[:2]
        day = date_str[2:]
        기준날짜 = f"2025-{month}-{day}"
        기준날짜 = pd.to_datetime(기준날짜).strftime('%Y-%m-%d')  # 날짜 포맷 보정
    except Exception as e:
        print(f"파일명에서 날짜 추출 실패: {file}, 에러: {e}")
        continue

    # '신청일자' 컬럼도 strip 처리
    if '신청일자' in df.columns:
        df['신청일자'] = df['신청일자'].astype(str).str.strip()
        df = df[df['신청일자'] == 기준날짜]
    else:
        print(f"파일 {file}에 '신청일자' 컬럼이 없습니다.")
        continue

    개수 = df.shape[0]
    날짜_개수_리스트.append({'날짜': 기준날짜, '개수': 개수})

# 리스트를 데이터프레임으로 변환
result_df = pd.DataFrame(날짜_개수_리스트)

# 엑셀로 저장
result_df.to_excel('신청일자별_개수_집계.xlsx', index=False)


Index(['지역구분', '차종구분', '신청번호', '지원가능여부요청번호', '승인번호', '공고구분', '신청단계', '신청일자',
       '계약일자', '자격부여일자', '지원가능요청일자', '대기자 선정유형', '승인일자', '출고기한', '지급신청일자',
       '지급신청기한', '지급일자', '취소(포기)일자', '출고예정일자', '신청유형', '신청구분', '기준연도', '차종',
       '신청대수', '신청자명', '생년월일(법인등록번호)', '성별', '사업자번호', '개인사업장명', '주소(등록주소지)',
       '우편번호', '전화번호', '휴대폰', '이메일', '신청총금액(만원)', '추가보조금(만원)', '택배차추가 보조금(만원)',
       '차량번호', '차대번호', '세금계산서(차량가격)(원)', '은행', '예금주', '계좌번호', '제조수입사관리번호',
       '대리점', '대리점 연락처', '작성자', '작성자연락처', '택시여부', '택시법인중소기업여부', '세대주명',
       '세대주생년월일', '공동명의성명', '공동명의생년월일', '사회계층여부', '사회계층유형', '내연기관폐차여부',
       '내연기관 폐차차량번호', '내연기관 폐차차대번호', '리스·렌탈 이용여부', '리스·렌탈 이용자성명(기관명)',
       '리스·렌탈 이용자생년월일/법인등록번호', '리스·렌탈 이용자사업자번호', '우선순위 선택',
       '미세먼지 개선효과가 높은 차량 구매자 여부', '차상위 이하계층 여부', '생애최초 차량구매자 여부',
       'K-EV100참여 여부', '친환경차 구매목표제대상기업 여부', 'BMS 업데이트 불가차량 폐차 후 신차 구매 여부'],
      dtype='object')
Index(['지역구분', '차종구분', '신청번호', '지원가능여부요청번호', '승인번호', '공고구분', '신청단계', '신청일자',
       '계약일자', '자격부여일자'

In [ ]:
result = pd.read_excel('신청일자별_개수_집계.xlsx')

folder_path = 'C:/Users/HP/Downloads/11'
except_list = ['승인불가', '신청포기', '지원취소']

날짜_개수_리스트 = []

for file in os.listdir(folder_path):
    if file.endswith('.xlsx'):
        df = pd.read_excel(os.path.join(folder_path, file))
        df = df[~df['신청단계'].isin(except_list)]

    elif file.endswith('.xls'):
        df = pd.read_excel(os.path.join(folder_path, file), header=2)
        df = df[~df['신청단계'].isin(except_list)]
        
    # '취소(포기)\n일자'가 기준날짜인 경우
    date_str = file.split()[1]  # 예: '0724' 또는 '0624' 추출
    month = date_str[:2]
    day = date_str[2:]
    기준날짜 = f"2025-{month}-{day}"
    기준날짜 = pd.to_datetime(기준날짜).strftime('%Y-%m-%d')  # 날짜 포맷 보정
    df = df[df['신청일자'] == 기준날짜]
    
    개수 = df.shape[0]
    날짜_개수_리스트.append({'날짜': 기준날짜, '개수': 개수})

# 새로 구한 날짜별 개수 데이터프레임 생성
new_df = pd.DataFrame(날짜_개수_리스트)

# 기존 result와 합치기 (중복 날짜는 모두 남김)
result = pd.concat([result, new_df], ignore_index=True)
result.to_excel('신청일자별_개수_집계.xlsx', index=False)

In [ ]:

folder_path = 'C:/Users/HP/Downloads/11'
except_list = ['승인불가', '신청포기', '지원취소']

날짜_개수_리스트 = []

for file in os.listdir(folder_path):
    if file.endswith('.xlsx'):
        df = pd.read_excel(os.path.join(folder_path, file))
        df = df[~df['신청단계'].isin(except_list)]

    elif file.endswith('.xls'):
        df = pd.read_excel(os.path.join(folder_path, file), header=2)
        df = df[~df['신청단계'].isin(except_list)]
        
    # '취소(포기)\n일자'가 기준날짜인 경우
    date_str = file.split()[1]  # 예: '0724' 또는 '0624' 추출
    month = date_str[:2]
    day = date_str[2:]
    기준날짜 = f"2025-{month}-{day}"
    기준날짜 = pd.to_datetime(기준날짜).strftime('%Y-%m-%d')  # 날짜 포맷 보정
    df = df[df['신청일자'] == 기준날짜]
    
    개수 = df.shape[0]
    날짜_개수_리스트.append({'날짜': 기준날짜, '개수': 개수})

# 새로 구한 날짜별 개수 데이터프레임 생성
new_df = pd.DataFrame(날짜_개수_리스트)

# 기존 result와 합치기 (중복 날짜는 모두 남김)
result = pd.concat([result, new_df], ignore_index=True)
result.to_excel('신청일자별_개수_집계.xlsx', index=False)

In [50]:
result = pd.read_excel('신청일자별_개수_집계.xlsx')
folder_path = 'C:/Users/HP/Downloads/33'
except_list = ['승인불가', '신청포기', '지원취소']

날짜_개수_리스트 = []

for file in os.listdir(folder_path):
    if file.endswith('.xlsx'):
        df = pd.read_excel(os.path.join(folder_path, file))
        df = df[~df['신청단계'].isin(except_list)]

    elif file.endswith('.xls'):
        df = pd.read_excel(os.path.join(folder_path, file), header=2)
        df = df[~df['신청단계'].isin(except_list)]
        
    # '취소(포기)\n일자'가 기준날짜인 경우
    date_str = file.split()[1]  # 예: '0724' 또는 '0624' 추출
    month = date_str[:2]
    day = date_str[2:]
    기준날짜 = f"2025-{month}-{day}"
    기준날짜 = pd.to_datetime(기준날짜).strftime('%Y-%m-%d')  # 날짜 포맷 보정
    df = df[df['신청일자'] == 기준날짜]
    
    개수 = df.shape[0]
    날짜_개수_리스트.append({'날짜': 기준날짜, '개수': 개수})

# 새로 구한 날짜별 개수 데이터프레임 생성
new_df = pd.DataFrame(날짜_개수_리스트)

# 기존 result와 합치기 (중복 날짜는 모두 남김)
result = pd.concat([result, new_df], ignore_index=True)
result.to_excel('신청일자별_개수_집계.xlsx', index=False)

In [53]:
folder_path = 'C:/Users/HP/Downloads/22'
except_list = ['승인불가', '신청포기', '지원취소']

기준날짜_취소_개수_리스트 = []

for file in os.listdir(folder_path):
    if file.endswith('.xlsx'):
        df = pd.read_excel(os.path.join(folder_path, file))
        df = df[df['신청단계'].isin(except_list)]
    elif file.endswith('.xls'):
        df = pd.read_excel(os.path.join(folder_path, file), header=2)
        df = df[df['신청단계'].isin(except_list)]

    # '취소(포기)\n일자'가 기준날짜인 경우
    date_str = file.split()[1]  # 예: '0724' 또는 '0624' 추출
    month = date_str[:2]
    day = date_str[2:]
    기준날짜 = f"2025-{month}-{day}"
    기준날짜 = pd.to_datetime(기준날짜).strftime('%Y-%m-%d')  # 날짜 포맷 보정
    df = df[df['취소(포기)\n일자'] == 기준날짜]

    # 각 신청단계별 개수 계산
    승인불가_개수 = df[df['신청단계'] == '승인불가'].shape[0]
    신청포기_개수 = df[df['신청단계'] == '신청포기'].shape[0]
    지원취소_개수 = df[df['신청단계'] == '지원취소'].shape[0]
    
    # 전체 개수 (세 개의 합계)
    전체_개수 = 승인불가_개수 + 신청포기_개수 + 지원취소_개수
    
    기준날짜_취소_개수_리스트.append({
        '날짜': 기준날짜, 
        '개수': 전체_개수,
        '승인불가': 승인불가_개수,
        '신청포기': 신청포기_개수,
        '지원취소': 지원취소_개수
    })

new_df = pd.DataFrame(기준날짜_취소_개수_리스트)
new_df.to_excel('기준날짜_취소_개수_집계.xlsx', index=False)

In [54]:
df = pd.read_excel('Greet_Subsidy_2Q.xlsx', sheet_name='지급신청', header=3)

# 배분일별 개수 집계 및 데이터프레임화
배분일_개수 = df['배분일'].value_counts().reset_index()
배분일_개수.columns = ['날짜', '개수']

# 날짜 오름차순 정렬
배분일_개수 = 배분일_개수.sort_values('날짜').reset_index(drop=True)

# 엑셀로 저장
배분일_개수.to_excel('배분일별_개수_집계.xlsx', index=False)

In [55]:
df = pd.read_excel('Greet_Subsidy_2Q.xlsx', sheet_name='지급신청', header=3)

# 배분일별 개수 집계 및 데이터프레임화
배분일_개수 = df['지급\n신청일'].value_counts().reset_index()
배분일_개수.columns = ['날짜', '개수']

# 날짜 오름차순 정렬
배분일_개수 = 배분일_개수.sort_values('날짜').reset_index(drop=True)

# 엑셀로 저장
배분일_개수.to_excel('지급일별_개수_집계.xlsx', index=False)

In [56]:
df = pd.read_excel('Greet_Subsidy_2Q.xlsx', sheet_name='지급신청', header=3)
df_ev = pd.read_excel('EV_Q2.xlsx')

배분일_rn = df[['배분일', 'RN']]
지급신청일_rn = df_ev[['지급신청일자', '제조수입사\n관리번호']]

# 날짜 컬럼을 datetime으로 변환 후 날짜만 추출
배분일_rn['배분일'] = pd.to_datetime(배분일_rn['배분일'], errors='coerce').dt.strftime('%Y-%m-%d')
지급신청일_rn['지급신청일자'] = pd.to_datetime(지급신청일_rn['지급신청일자'], errors='coerce').dt.strftime('%Y-%m-%d')

# 'RN'과 '제조수입사\n관리번호'를 기준으로 left join
merged_df = 배분일_rn.merge(
    지급신청일_rn, 
    left_on='RN', 
    right_on='제조수입사\n관리번호', 
    how='left'
)

# 결과 컬럼만 남기기
result_df = merged_df[['RN', '배분일', '지급신청일자']]

result_df.to_excel('지급_집계.xlsx', index=False)


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_7948\3672846104.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  배분일_rn['배분일'] = pd.to_datetime(배분일_rn['배분일'], errors='coerce').dt.strftime('%Y-%m-%d')
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_7948\3672846104.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  지급신청일_rn['지급신청일자'] = pd.to_datetime(지급신청일_rn['지급신청일자'], errors='coerce').dt.strftime('%Y-%m-%d')


In [ ]:
import pandas as pd

df = pd.read_excel('2025년 테슬라 EV추출파일.xlsx')

df_sample = df[['지역구분', '주소\n(등록주소지)']]

## df_sample을 엑셀로 저장
df_sample.to_excel('sample.xlsx', index=False)